In [1]:
import requests
import json
import os
from time import sleep
API_KEY = {'X-API-Key': 'KZJ94OPT'}
session = requests.Session()
session.headers.update(API_KEY)

In [2]:
class ApiException(Exception):
    pass

class HelperFunctions():
    
    def __init__(self):
        pass

    def ticker_bid_ask(self, session, ticker):
        payload = {'ticker': ticker}
        resp = session.get('http://localhost:9999/v1/securities/book', params=payload)  
        if resp.ok:
            book = resp.json()
            return book['bids'][0]['price'], book['asks'][0]['price']
        raise ApiException('Error getting bid / ask: The API key provided in this Python code must match that in the RIT client')

In [3]:
def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.status_code == 401:
        raise ApiException('Error getting tick: The API key provided in this Python code must match that in the RIT client')
    case = resp.json()
    return case['tick']

In [16]:
assets = session.get('http://localhost:9999/v1/assets')
assets.json()

[{'ticker': 'CL-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil Spot in Cushing',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert_from': [],
  'convert_to': [],
  'containment': {'ticker': 'CL', 'quantity': 10.0},
  'ticks_per_conversion': 0,
  'ticks_per_lease': 30,
  'is_available': True,
  'start_period': 1,
  'stop_period': 2},
 {'ticker': 'AK-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil in Alaska',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert_from': [],
  'convert_to': [],
  'containment': {'ticker': 'CL-AK', 'quantity': 10.0},
  'ticks_per_conversion': 0,
  'ticks_per_lease': 30,
  'is_available': True,
  'start_period': 1,
  'stop_period': 2},
 {'ticker': 'NYC-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil in New York City',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert

In [15]:
x = session.post('http://localhost:9999/v1/leases/AK-CS-PIPE?from1=CL&quantity1=30')

{'code': 'NOT_FOUND', 'message': 'Requested route POST:/v1/leases/AK-CS-PIPE was not found.'}


In [17]:
x = session.post('http://localhost:9999/v1/leases', params = {'ticker': 'AK-CS-PIPE'})
print(x.json())

{'code': 'INTERNAL_SERVER_ERROR', 'message': 'Only CONTAINER, REFINERY, and POWER_PLANT type assets can be leased.'}


In [21]:
x = session.post('http://localhost:9999/v1/leases?ticker=AK-CS-PIPE&from1=CL-AK&quantity1=30')
x.json()


{'code': 'INTERNAL_SERVER_ERROR',
 'message': 'Asset cannot be used on 30 units of CL-AK.'}

In [10]:
x = session.get('http://localhost:9999/v1/assets')
x.json()

[{'ticker': 'CL-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil Spot in Cushing',
  'total_quantity': 1000,
  'available_quantity': 997,
  'lease_price': 500.0,
  'convert_from': [],
  'convert_to': [],
  'containment': {'ticker': 'CL', 'quantity': 10.0},
  'ticks_per_conversion': 0,
  'ticks_per_lease': 30,
  'is_available': True,
  'start_period': 1,
  'stop_period': 2},
 {'ticker': 'AK-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil in Alaska',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert_from': [],
  'convert_to': [],
  'containment': {'ticker': 'CL-AK', 'quantity': 10.0},
  'ticks_per_conversion': 0,
  'ticks_per_lease': 30,
  'is_available': True,
  'start_period': 1,
  'stop_period': 2},
 {'ticker': 'NYC-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil in New York City',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert_

In [16]:
def ticker_close(session, ticker):
    payload = {'ticker': ticker, 'limit': 1}
    resp = session.get('http://localhost:9999/v1/securities/history', params=payload)
    if resp.status_code == 401:
        raise ApiException('The API key provided in this Python code must match that in the RIT client (please refer to the API hyperlink in the client toolbar and/or the RIT – User Guide – REST API Documentation.pdf)')
    ticker_history = resp.json()
    if ticker_history:
        return ticker_history[0]['close']
    else:
        raise ApiException('Response error. Unexpected JSON response.')

assets = session.get('http://localhost:9999/v1/assets')
assets = assets.json()

ak_pipe_price = 0
nyc_pipe_price = 0

# get pipeline price
for i in assets:
    if i['ticker'] == 'AK-CS-PIPE':
        ak_pipe_price = i['lease_price']
    if i['ticker'] == 'CS-NYC-PIPE':
        nyc_pipe_price = i['lease_price']

# get oil prices
cl_price = ticker_close(session, 'CL')
cl_ak_price = ticker_close(session, 'CL-AK')
cl_nyc_price = ticker_close(session, 'CL-NYC')

get_cl_1f = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-1F'})
cl_1f_price = get_cl_1f.json()[0]['ask']
cl_2f_price = ticker_close(session, 'CL-2F')

if cl_nyc_price > cl_price - nyc_pipe_price:
    
    #buy storage
    for i in range(5):
        session.post('http://localhost:9999/v1/leases', params = {'ticker': 'CL-STORAGE', 'from': 'CONTAINER'})
        sleep(.1)

    # BUY OIL
    session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL', 'type': 'MARKET', 'quantity': 50, 'action': 'BUY'})

    # BUY FUTURE
    if cl_1f_price > 0:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': 50, 'action': 'SELL'})
    else:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': 50, 'action': 'SELL'})

    # BUY PIPELINE
    session.post('http://localhost:9999/v1/leases', params = {'ticker': 'CS-NYC-PIPE', 'from': 'PIPELINE'})

    # USE PIPELINE
    leases = session.get('http://localhost:9999/v1/leases')

    pipeline_id = -1

    for i in leases.json():
        if i['ticker'] == 'CS-NYC-PIPE':
            pipeline_id = i['id']

    if pipeline_id == -1:
        session.post('http://localhost:9999/v1/leases', params = {'ticker': 'CS-NYC-PIPE', 'from':'PIPELINE', 'quantity': 50})
        for i in leases.json():
            if i['ticker'] == 'CS-NYC-PIPE':
                pipeline_id = i['id']

    session.post(f'http://localhost:9999/v1/leases/{pipeline_id}?from1=CL&quantity1=50')

    sleep(31)

    # SELL NYC AND FUTURE POSITION

    # cl_nyc = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-NYC'})
    # cl_nyc_position = cl_nyc.json()[0]['position']
    # if cl_nyc_position

    session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL_NYC', 'type': 'MARKET', 'quantity': 50, 'action': 'SELL'})
    
    cl_1f = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-1F'})
    cl_1f_position = cl_1f.json()[0]['position']
    cl_2f = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-2F'})
    cl_2f_position = cl_2f.json()[0]['position']

    if cl_1f_position > 0:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': cl_1f_position, 'action': 'BUY'})
    if cl_2f_position > 0:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': cl_2f_position, 'action': 'BUY'})

if cl_price > cl_ak_price - ak_pipe_price:
    
    #buy storage
    for i in range(5):
        session.post('http://localhost:9999/v1/leases', params = {'ticker': 'AK-STORAGE', 'from': 'CONTAINER'})
        sleep(.1)

    # BUY OIL
    session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-AK', 'type': 'MARKET', 'quantity': 50, 'action': 'BUY'})

    # BUY FUTURE
    if cl_1f_price > 0:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': 50, 'action': 'SELL'})
    else:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': 50, 'action': 'SELL'})

    # BUY PIPELINE
    session.post('http://localhost:9999/v1/leases', params = {'ticker': 'AK-CS-PIPE', 'from': 'PIPELINE'})

    # USE PIPELINE
    leases = session.get('http://localhost:9999/v1/leases')

    pipeline_id = -1

    for i in leases.json():
        if i['ticker'] == 'AK-CS-PIPE':
            pipeline_id = i['id']

    if pipeline_id == -1:
        session.post('http://localhost:9999/v1/leases', params = {'ticker': 'AK-CS-PIPE', 'from':'PIPELINE', 'quantity': 50})
        for i in leases.json():
            if i['ticker'] == 'AK-CS-PIPE':
                pipeline_id = i['id']

    session.post(f'http://localhost:9999/v1/leases/{pipeline_id}?from1=CL&quantity1=50')

    sleep(31)

    # SELL NYC AND FUTURE POSITION

    # cl_nyc = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-NYC'})
    # cl_nyc_position = cl_nyc.json()[0]['position']
    # if cl_nyc_position

    session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL', 'type': 'MARKET', 'quantity': 50, 'action': 'SELL'})
    
    cl_1f = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-1F'})
    cl_1f_position = cl_1f.json()[0]['position']
    cl_2f = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-2F'})
    cl_2f_position = cl_2f.json()[0]['position']

    if cl_1f_position > 0:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': cl_1f_position, 'action': 'BUY'})
    if cl_2f_position > 0:
        session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': cl_2f_position, 'action': 'BUY'})

In [12]:
get_cl_1f = session.get('http://localhost:9999/v1/securities', params = {'ticker': 'CL-1F'})
get_cl_1f.json()

[{'ticker': 'CL-1F',
  'type': 'FUTURE',
  'size': 1000,
  'position': 0.0,
  'vwap': 0.0,
  'nlv': 0.0,
  'last': 100.77,
  'bid': 0.0,
  'bid_size': 0.0,
  'ask': 0.0,
  'ask_size': 0.0,
  'volume': 0.0,
  'unrealized': 0.0,
  'realized': 0.0,
  'currency': '',
  'total_volume': 0.0,
  'limits': [{'name': 'CRUDE', 'units': 1.0}],
  'interest_rate': 0.0,
  'is_tradeable': False,
  'is_shortable': True,
  'start_period': 1,
  'stop_period': 1,
  'description': 'Crude Future at End of Month 1',
  'unit_multiplier': 1000,
  'display_unit': 'Barrels',
  'start_price': 101.0,
  'min_price': 0.0,
  'max_price': 1000.0,
  'quoted_decimals': 2,
  'trading_fee': 1.0,
  'limit_order_rebate': 0.0,
  'min_trade_size': 0,
  'max_trade_size': 30,
  'required_tickers': None,
  'underlying_tickers': None,
  'bond_coupon': 0.0,
  'interest_payments_per_period': 0,
  'base_security': '',
  'fixing_ticker': None,
  'api_orders_per_second': 100,
  'execution_delay_ms': 0,
  'interest_rate_ticker': None,


In [ ]:
refinery_id = 0

            for i in leases.json():
                if i['ticker'] == 'CL-REFINERY':
                    refinery_id = i['id']

            if refinery_id == 0:
                s.post('http://localhost:9999/v1/leases', params = {'ticker': 'CL-REFINERY', 'from':'CONTAINER', 'quantity': quantity})
                for i in leases.json():
                    if i['ticker'] == 'CL-REFINERY':
                        refinery_id = i['id']

            s.post(f'http://localhost:9999/v1/leases/{refinery_id}?from1=CL&quantity1={quantity}')

1. get pipeline prices
2. get oil prices
3. create strategy rule
4. hedge oil profits with futures
5. CREATE CLAUSE TO LIQUIDATE FUTURES AT TICK 598